In [4]:
import psycopg2

# 1. Connect to database
try:
    conn = psycopg2.connect(
        dbname="urban_data",
        user="postgres",
        password="12345",
        host="localhost",
        port="5432"
    )
    print("Connection successful!")

    # 2. Create a 'Cursor'
    cur = conn.cursor()

    # 3. Test it: Get the count of rows
    cur.execute("SELECT COUNT(*) FROM homicide_data;")
    count = cur.fetchone()
    print(f"I found {count[0]} rows in your database!")

except Exception as e:
    print(f"Error connecting to database: {e}")

Connection successful!
I found 638454 rows in your database!


In [5]:
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

geolocator = Nominatim(user_agent="mason_watch_v2")
geocode = RateLimiter(geolocator.geocode, min_delay_seconds=1)

# 3. Get a list of unique cities that need coordinates
cur.execute("SELECT DISTINCT city, state FROM homicide_data WHERE latitude IS NULL;")
locations = cur.fetchall()

print(f"Found {len(locations)} cities to geocode...")

for city, state in locations:
    address = f"{city}, {state}, USA"
    try:
        location = geolocator.geocode(address)
        if location:
            print(f"Found: {address} at {location.latitude}, {location.longitude}")

            # 4. Save the coordinates back to the database
            cur.execute("""
                UPDATE homicide_data
                SET latitude = %s, longitude = %s
                WHERE city = %s AND state = %s
            """, (location.latitude, location.longitude, city, state))
            #Save coordinates to the DB
            conn.commit()
    except Exception as e:
        print(f"Error geocoding {address}: {e}")

cur.close()
conn.close()
print("Batch complete!")

Found 2992 cities to geocode...
Found: Prowers, Colorado, USA at 37.9455797, -102.4130793
Found: Wright, Minnesota, USA at 45.1489061, -93.9639196
Found: Klamath, Oregon, USA at 42.6052565, -121.739544
Found: Will, Illinois, USA at 41.4194058, -87.9994754
Found: Warren, Tennessee, USA at 35.6889753, -85.7796626
Found: Throckmorton, Texas, USA at 33.1571081, -99.2017838
Found: Mills, Iowa, USA at 41.0171392, -95.6087477
Found: Saginaw, Michigan, USA at 43.4200387, -83.9490365
Found: Stevens, Washington, USA at 48.4249969, -117.8489411
Found: Stone, Missouri, USA at 36.7377156, -93.4555059
Found: Maury, Tennessee, USA at 35.6285992, -87.0736684
Found: Rogers, Oklahoma, USA at 36.3781607, -95.6190889
Found: Fresno, California, USA at 36.7394421, -119.78483
Found: Adams, Wisconsin, USA at 43.9462246, -89.7836456
Found: Sully, South Dakota, USA at 44.7173891, -100.0690149
Found: Van Buren, Arkansas, USA at 35.5759086, -92.4916144
Found: Effingham, Georgia, USA at 32.3419626, -81.338687
Foun